In [ ]:
#|default_exp forecast

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

# MLForecast

> Full pipeline encapsulation

In [ ]:
#|export
import copy
import re
import warnings
from typing import TYPE_CHECKING, Callable, Dict, Iterable, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, clone
from utilsforecast.compat import DataFrame
from utilsforecast.processing import (
    assign_columns,
    backtest_splits,
    copy_if_pandas,
    counts_by_id,
    drop_index_if_pandas,
    filter_with_mask,
    is_in,
    is_nan,
    join,
    maybe_compute_sort_indices,
    take_rows,
    to_numpy,
    vertical_concat,
)

from mlforecast.core import (
    DateFeature,
    Freq,
    LagTransforms,
    Lags,
    Models,
    TargetTransform,
    TimeSeries,
    _name_models,
)
from mlforecast.grouped_array import GroupedArray
if TYPE_CHECKING:
    from mlforecast.lgb_cv import LightGBMCV
from mlforecast.target_transforms import BaseGroupedArrayTargetTransform
from mlforecast.utils import PredictionIntervals

In [ ]:
#|hide
from fastcore.test import test_warns, test_eq, test_ne, test_fail, test_close
from nbdev import show_doc
from sklearn import set_config

In [ ]:
#|hide
set_config(display='text')
warnings.simplefilter('ignore', UserWarning)

In [ ]:
#| exporti
def _add_conformal_distribution_intervals(
        fcst_df: DataFrame, 
        cs_df: DataFrame,
        model_names: List[str],
        level: List[Union[int, float]],
        cs_n_windows: int,
        cs_h: int,
        n_series: int,
        horizon: int,
    ) -> DataFrame:
    """
    Adds conformal intervals to a `fcst_df` based on conformal scores `cs_df`.
    `level` should be already sorted. This strategy creates forecasts paths
    based on errors and calculate quantiles using those paths.
    """
    fcst_df = copy_if_pandas(fcst_df, deep=False)
    alphas = [100 - lv for lv in level]
    cuts = [alpha / 200 for alpha in reversed(alphas)]
    cuts.extend(1 - alpha / 200 for alpha in alphas)
    for model in model_names:
        scores = cs_df[model].to_numpy().reshape(cs_n_windows, n_series, cs_h)
        # restrict scores to horizon
        scores = scores[:,:,:horizon]
        mean = fcst_df[model].to_numpy().reshape(1, n_series, -1)
        scores = np.vstack([mean - scores, mean + scores])
        quantiles = np.quantile(
            scores,
            cuts,
            axis=0,
        )
        quantiles = quantiles.reshape(len(cuts), -1).T
        lo_cols = [f"{model}-lo-{lv}" for lv in reversed(level)]
        hi_cols = [f"{model}-hi-{lv}" for lv in level]
        out_cols = lo_cols + hi_cols
        fcst_df = assign_columns(fcst_df, out_cols, quantiles)
    return fcst_df

In [ ]:
#| exporti
def _add_conformal_error_intervals(
        fcst_df: DataFrame, 
        cs_df: DataFrame, 
        model_names: List[str],
        level: List[Union[int, float]],
        cs_n_windows: int,
        cs_h: int,
        n_series: int,
        horizon: int,
    ) -> DataFrame:
    """
    Adds conformal intervals to a `fcst_df` based on conformal scores `cs_df`.
    `level` should be already sorted. This startegy creates prediction intervals
    based on the absolute errors.
    """
    fcst_df = copy_if_pandas(fcst_df, deep=False)
    cuts = [lv / 100 for lv in level]
    for model in model_names:
        mean = fcst_df[model].to_numpy().ravel()
        scores = cs_df[model].to_numpy().reshape(cs_n_windows, n_series, cs_h)
        # restrict scores to horizon
        scores = scores[:,:,:horizon]
        quantiles = np.quantile(
            scores,
            cuts,
            axis=0,
        )
        quantiles = quantiles.reshape(len(cuts), -1)
        lo_cols = [f"{model}-lo-{lv}" for lv in reversed(level)]
        hi_cols = [f"{model}-hi-{lv}" for lv in level]
        quantiles = np.vstack([mean - quantiles[::-1], mean + quantiles]).T
        columns = lo_cols + hi_cols
        fcst_df = assign_columns(fcst_df, columns, quantiles)
    return fcst_df

In [ ]:
#| exporti
def _get_conformal_method(method: str):
    available_methods = {
        'conformal_distribution': _add_conformal_distribution_intervals,
        'conformal_error': _add_conformal_error_intervals, 
    }
    if method not in available_methods.keys():
        raise ValueError(
            f'prediction intervals method {method} not supported '
            f'please choose one of {", ".join(available_methods.keys())}'
        )
    return available_methods[method]

In [ ]:
#| hide
test_fail(lambda: _get_conformal_method('my_method'))

In [ ]:
#|export
class MLForecast:
    def __init__(
        self,
        models: Models,
        freq: Freq,
        lags: Optional[Lags] = None,
        lag_transforms: Optional[LagTransforms] = None,
        date_features: Optional[Iterable[DateFeature]] = None,
        num_threads: int = 1,
        target_transforms: Optional[List[TargetTransform]] = None,
        transforms_namer: Optional[Callable] = None,
    ):
        """Forecasting pipeline

        Parameters
        ----------
        models : regressor or list of regressors
            Models that will be trained and used to compute the forecasts.
        freq : str or int or pd.offsets.BaseOffset
            Pandas offset, pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series.
        lags : list of int, optional (default=None)
            Lags of the target to use as features.
        lag_transforms : dict of int to list of functions, optional (default=None)
            Mapping of target lags to their transformations.
        date_features : list of str or callable, optional (default=None)
            Features computed from the dates. Can be pandas date attributes or functions that will take the dates as input.
        num_threads : int (default=1)
            Number of threads to use when computing the features.
        target_transforms : list of transformers, optional(default=None)
            Transformations that will be applied to the target before computing the features and restored after the forecasting step.
        transforms_namer : callable, optional(default=None)
            Function that takes a transformation (either function or class), a lag and extra arguments and produces a name.
        """
        if not isinstance(models, dict) and not isinstance(models, list):
            models = [models]
        if isinstance(models, list):
            model_names = _name_models([m.__class__.__name__ for m in models])            
            models_with_names = dict(zip(model_names, models))
        else:
            models_with_names = models
        self.models = models_with_names
        self.ts = TimeSeries(
            freq=freq,
            lags=lags,
            lag_transforms=lag_transforms,
            date_features=date_features,
            num_threads=num_threads,
            target_transforms=target_transforms,
            transforms_namer=transforms_namer,
        )
        
    def __repr__(self):
        return (
            f'{self.__class__.__name__}(models=[{", ".join(self.models.keys())}], '
            f'freq={self.freq}, '
            f'lag_features={list(self.ts.transforms.keys())}, '
            f'date_features={self.ts.date_features}, '
            f'num_threads={self.ts.num_threads})'
        )
    
    @property
    def freq(self):
        return self.ts.freq

    @classmethod
    def from_cv(cls, cv: 'LightGBMCV') -> 'MLForecast':
        if not hasattr(cv, 'best_iteration_'):
            raise ValueError('LightGBMCV object must be fitted first.')
        import lightgbm as lgb
        fcst = cls(
            models=lgb.LGBMRegressor(**{**cv.params, 'n_estimators': cv.best_iteration_}),
            freq=cv.ts.freq,
        )
        fcst.ts = copy.deepcopy(cv.ts)
        return fcst

    def preprocess(
        self,
        df: DataFrame,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        max_horizon: Optional[int] = None,
        return_X_y: bool = False,
        as_numpy: bool = False,
    ) -> Union[DataFrame, Tuple[DataFrame, np.ndarray]]:
        """Add the features to `data`.
        
        Parameters
        ----------
        df : pandas DataFrame
            Series data in long format.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        static_features : list of str, optional (default=None)
            Names of the features that are static and will be repeated when forecasting.
        dropna : bool (default=True)
            Drop rows with missing values produced by the transformations.
        keep_last_n : int, optional (default=None)
            Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.
        max_horizon : int, optional (default=None)
            Train this many models, where each model will predict a specific horizon.
        return_X_y : bool (default=False)
            Return a tuple with the features and the target. If False will return a single dataframe.
        as_numpy : bool (default = False)
            Cast features to numpy array. Only works for `return_X_y=True`.

        Returns
        -------
        result : DataFrame or tuple of pandas Dataframe and a numpy array.
            `df` plus added features and target(s).
        """     
        return self.ts.fit_transform(
            df,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
            max_horizon=max_horizon,
            return_X_y=return_X_y,
            as_numpy=as_numpy,
        )

    def fit_models(
        self,
        X: Union[DataFrame, np.ndarray],
        y: np.ndarray,
    ) -> 'MLForecast':
        """Manually train models. Use this if you called `MLForecast.preprocess` beforehand.
        
        Parameters
        ----------
        X : pandas or polars DataFrame or numpy array
            Features.
        y : numpy array.
            Target.

        Returns
        -------
        self : MLForecast
            Forecast object with trained models.
        """
        self.models_: Dict[str, Union[BaseEstimator, List[BaseEstimator]]] = {}
        for name, model in self.models.items():
            if y.ndim == 2 and y.shape[1] > 1:
                self.models_[name] = []                
                for col in range(y.shape[1]):
                    keep = ~np.isnan(y[:, col])
                    if isinstance(X, np.ndarray):
                        # TODO: migrate to utils
                        Xh = X[keep]
                    else:
                        Xh = filter_with_mask(X, keep)
                    yh = y[keep, col]
                    self.models_[name].append(clone(model).fit(Xh, yh))
            else:
                self.models_[name] = clone(model).fit(X, y)
        return self

    def _conformity_scores(
        self,
        df: DataFrame,
        id_col: str, 
        time_col: str,
        target_col: str,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        max_horizon: Optional[int] = None,
        n_windows: int = 2,
        h: int = 1,
        as_numpy: bool = False,
    ):
        """Compute conformity scores.
        
        We need at least two cross validation errors to compute
        quantiles for prediction intervals (`n_windows=2`).
        
        The exception is raised by the PredictionIntervals data class.
        
        In this simplest case, we assume the width of the interval
        is the same for all the forecasting horizon (`h=1`).
        """
        cv_results = self.cross_validation(
            df=df, 
            n_windows=n_windows,
            h=h,
            refit=False,
            id_col=id_col, 
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
            max_horizon=max_horizon,
            prediction_intervals=None,
            as_numpy=as_numpy,
        )
        # conformity score for each model
        for model in self.models.keys():
            # compute absolute error for each model
            abs_err = abs(cv_results[model] - cv_results[target_col])
            cv_results = assign_columns(cv_results, model, abs_err)
        return cv_results.drop(columns=target_col)

    def _invert_transforms_fitted(self, df: pd.DataFrame) -> pd.DataFrame:
        if self.ts.target_transforms is None:
            return df
        if any(isinstance(tfm, BaseGroupedArrayTargetTransform) for tfm in self.ts.target_transforms):
            model_cols = [c for c in df.columns if c not in (self.ts.id_col, self.ts.time_col)]
            id_counts = counts_by_id(df, self.ts.id_col)
            sizes = id_counts['counts'].to_numpy()
            indptr = np.append(0, sizes.cumsum())
        for tfm in self.ts.target_transforms[::-1]:
            if isinstance(tfm, BaseGroupedArrayTargetTransform):
                if self.ts._dropped_series is not None:
                    tfm.idxs = np.delete(np.arange(self.ts.ga.n_groups), self.ts._dropped_series)
                for col in model_cols:
                    ga = GroupedArray(df[col].to_numpy(), indptr)
                    ga = tfm.inverse_transform_fitted(ga)
                    df = assign_columns(df, col, ga.data)
                tfm.idxs = None
            else:
                df = tfm.inverse_transform(df)
        return df

    def _extract_X_y(
        self,
        prep: DataFrame,
        target_col: str,
    ) -> Tuple[Union[DataFrame, np.ndarray], np.ndarray]:
        X = prep[self.ts.features_order_]
        targets = [c for c in prep.columns if re.match(rf'^{target_col}\d?$', c)]
        if len(targets) == 1:
            targets = targets[0]
        y = prep[targets].to_numpy()        
        return X, y

    def _compute_fitted_values(
        self,
        base: DataFrame,
        X: Union[DataFrame, np.ndarray],
        y: np.ndarray,
        id_col: str,
        time_col: str,
        target_col: str,
        max_horizon: Optional[int],
    ) -> DataFrame:
        base = copy_if_pandas(base, deep=False)
        sort_idxs = maybe_compute_sort_indices(base, id_col, time_col)
        if sort_idxs is not None:
            base = take_rows(base, sort_idxs)
            X = take_rows(X, sort_idxs)
            y = y[sort_idxs]
        if max_horizon is None:
            fitted_values = assign_columns(base, target_col, y)
            for name, model in self.models_.items():
                assert not isinstance(model, list)  # mypy
                preds = model.predict(X)
                fitted_values = assign_columns(fitted_values, name, preds)
            fitted_values = self._invert_transforms_fitted(fitted_values)
        else:
            horizon_fitted_values = []
            for horizon in range(max_horizon):
                horizon_base = copy_if_pandas(base, deep=True)
                horizon_base = assign_columns(horizon_base, target_col, y[:, horizon])
                horizon_fitted_values.append(horizon_base)
            for name, horizon_models in self.models_.items():
                for horizon, model in enumerate(horizon_models):
                    preds = model.predict(X)
                    horizon_fitted_values[horizon] = assign_columns(
                        horizon_fitted_values[horizon], name, preds
                    )
            for horizon, horizon_df in enumerate(horizon_fitted_values):
                keep_mask = ~is_nan(horizon_df[target_col])
                horizon_df = filter_with_mask(horizon_df, keep_mask)
                horizon_df = copy_if_pandas(horizon_df, deep=True)
                horizon_df = self._invert_transforms_fitted(horizon_df)
                horizon_df = assign_columns(horizon_df, 'h', horizon + 1)
                horizon_fitted_values[horizon] = horizon_df
            fitted_values = vertical_concat(horizon_fitted_values, match_categories=False)
        if self.ts.target_transforms is not None:
            for tfm in self.ts.target_transforms[::-1]:            
                if hasattr(tfm, 'store_fitted'):
                    tfm.store_fitted = False
                if hasattr(tfm, 'fitted_'):
                    tfm.fitted_ = []            
        return fitted_values

    def fit(
        self,
        df: DataFrame,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        max_horizon: Optional[int] = None,
        prediction_intervals: Optional[PredictionIntervals] = None,
        fitted: bool = False,
        as_numpy: bool = False,
    ) -> 'MLForecast':
        """Apply the feature engineering and train the models.
        
        Parameters
        ----------
        df : pandas or polars DataFrame
            Series data in long format.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        static_features : list of str, optional (default=None)
            Names of the features that are static and will be repeated when forecasting.
                If `None`, will consider all columns (except id_col and time_col) as static.
        dropna : bool (default=True)
            Drop rows with missing values produced by the transformations.
        keep_last_n : int, optional (default=None)
            Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.
        max_horizon : int, optional (default=None)
            Train this many models, where each model will predict a specific horizon.
        prediction_intervals : PredictionIntervals, optional (default=None)
            Configuration to calibrate prediction intervals (Conformal Prediction).
        fitted : bool (default=False)
            Save in-sample predictions.
        as_numpy : bool (default = False)
            Cast features to numpy array.

        Returns
        -------
        self : MLForecast
            Forecast object with series values and trained models.
        """
        if fitted and self.ts.target_transforms is not None:
            for tfm in self.ts.target_transforms:
                if hasattr(tfm, 'store_fitted'):
                    tfm.store_fitted = True
        self._cs_df: Optional[DataFrame] = None
        if prediction_intervals is not None:
            self.prediction_intervals = prediction_intervals
            self._cs_df = self._conformity_scores(
                df=df, 
                id_col=id_col, 
                time_col=time_col,
                target_col=target_col,
                static_features=static_features,
                dropna=dropna,
                keep_last_n=keep_last_n,
                n_windows=prediction_intervals.n_windows,
                h=prediction_intervals.h,
                as_numpy=as_numpy,
            )
        prep = self.preprocess(
            df=df,
            id_col=id_col,
            time_col=time_col,
            target_col=target_col,
            static_features=static_features,
            dropna=dropna,
            keep_last_n=keep_last_n,
            max_horizon=max_horizon,
            return_X_y=not fitted,
            as_numpy=as_numpy,
        )
        if isinstance(prep, tuple):
            X, y = prep
        else:
            base = prep[[id_col, time_col]]
            X, y = self._extract_X_y(prep, target_col)
            if as_numpy:
                X = to_numpy(X)
            del prep
        self.fit_models(X, y)
        if fitted:
            fitted_values = self._compute_fitted_values(
                base=base,
                X=X,
                y=y,
                id_col=id_col,
                time_col=time_col,
                target_col=target_col,
                max_horizon=max_horizon,
            )
            fitted_values = drop_index_if_pandas(fitted_values)
            self.fcst_fitted_values_ = fitted_values
        return self

    def forecast_fitted_values(self):
        """Access in-sample predictions."""
        if not hasattr(self, 'fcst_fitted_values_'):
            raise Exception('Please run the `fit` method using `fitted=True`')
        return self.fcst_fitted_values_

    def predict(
        self,
        h: int,
        before_predict_callback: Optional[Callable] = None,
        after_predict_callback: Optional[Callable] = None,
        new_df: Optional[DataFrame] = None,
        level: Optional[List[Union[int, float]]] = None,
        X_df: Optional[DataFrame] = None,
        ids: Optional[List[str]] = None,
    ) -> DataFrame:
        """Compute the predictions for the next `h` steps.
        
        Parameters
        ----------
        h : int
            Number of periods to predict.
        before_predict_callback : callable, optional (default=None)
            Function to call on the features before computing the predictions.
                This function will take the input dataframe that will be passed to the model for predicting and should return a dataframe with the same structure.
                The series identifier is on the index.
        after_predict_callback : callable, optional (default=None)
            Function to call on the predictions before updating the targets.
                This function will take a pandas Series with the predictions and should return another one with the same structure.
                The series identifier is on the index.   
        new_df : pandas or polars DataFrame, optional (default=None)
            Series data of new observations for which forecasts are to be generated. 
                This dataframe should have the same structure as the one used to fit the model, including any features and time series data. 
                If `new_df` is not None, the method will generate forecasts for the new observations.
        level : list of ints or floats, optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        X_df : pandas or polars DataFrame, optional (default=None)
            Dataframe with the future exogenous features. Should have the id column and the time column.
        ids : list of str, optional (default=None)
            List with subset of ids seen during training for which the forecasts should be computed.        

        Returns
        -------
        result : pandas or polars DataFrame
            Predictions for each serie and timestep, with one column per model.
        """
        if not hasattr(self, 'models_'):
            raise ValueError(
                "No fitted models found. You have to call fit or preprocess + fit_models. "
                "If you used cross_validation before please fit again."
            )
        first_model_is_list = isinstance(next(iter(self.models_.values())), list)
        max_horizon = self.ts.max_horizon
        if first_model_is_list and max_horizon is None:
            raise ValueError(
                'Found one model per horizon but `max_horizon` is None. '
                'If you ran preprocess after fit please run fit again.'
            )
        elif not first_model_is_list and max_horizon is not None:
            raise ValueError(
                'Found a single model for all horizons '
                f'but `max_horizon` is {max_horizon}. '
                'If you ran preprocess after fit please run fit again.'
            )

        if new_df is not None:
            new_ts = TimeSeries(
                freq=self.ts.freq,
                lags=self.ts.lags, 
                lag_transforms=self.ts.lag_transforms,
                date_features=self.ts.date_features, 
                num_threads=self.ts.num_threads,
                target_transforms=self.ts.target_transforms,
            )
            new_ts._fit(
                new_df,
                id_col=self.ts.id_col,
                time_col=self.ts.time_col,
                target_col=self.ts.target_col, 
                static_features=self.ts.static_features,
                keep_last_n=self.ts.keep_last_n,
            )
            new_ts.max_horizon = self.ts.max_horizon
            new_ts.as_numpy = self.ts.as_numpy
            ts = new_ts
        else:
            ts = self.ts
            
        forecasts = ts.predict(
            models=self.models_,
            horizon=h,
            before_predict_callback=before_predict_callback,
            after_predict_callback=after_predict_callback,
            X_df=X_df,
            ids=ids,
        )
        if level is not None:
            if self._cs_df is None:
                warn_msg = (
                    'Please rerun the `fit` method passing a proper value '
                    'to prediction intervals to compute them.'
                )
                warnings.warn(warn_msg, UserWarning)
            else:
                if (self.prediction_intervals.h != 1) and (self.prediction_intervals.h < h):
                    raise ValueError(
                        'The `h` argument of PredictionIntervals '
                        'should be equal to one or greater or equal to `h`. '
                        'Please rerun the `fit` method passing a proper value '
                        'to prediction intervals.'
                    )
                if self.prediction_intervals.h == 1 and h > 1:
                    warn_msg = (
                        'Prediction intervals are calculated using 1-step ahead cross-validation, '
                        'with a constant width for all horizons. To vary the error by horizon, '
                        'pass PredictionIntervals(h=h) to the `prediction_intervals` '
                        'argument when refitting the model.'
                    )
                    warnings.warn(warn_msg, UserWarning)
                level_ = sorted(level)
                model_names = self.models.keys()
                conformal_method = _get_conformal_method(self.prediction_intervals.method)
                if ids is not None:
                    ids_mask = is_in(self._cs_df[self.ts.id_col], ids)
                    cs_df = filter_with_mask(self._cs_df, ids_mask)
                    n_series = len(ids)
                else:
                    cs_df = self._cs_df
                    n_series = self.ts.ga.n_groups
                forecasts = conformal_method(
                    forecasts,
                    cs_df,
                    model_names=list(model_names),
                    level=level_,
                    cs_h=self.prediction_intervals.h,
                    cs_n_windows=self.prediction_intervals.n_windows,
                    n_series=n_series,
                    horizon=h,
                )
        return forecasts

    def cross_validation(
        self,
        df: DataFrame,
        n_windows: int,
        h: int,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
        step_size: Optional[int] = None,
        static_features: Optional[List[str]] = None,
        dropna: bool = True,
        keep_last_n: Optional[int] = None,
        refit: Union[bool, int] = True,
        max_horizon: Optional[int] = None,
        before_predict_callback: Optional[Callable] = None,
        after_predict_callback: Optional[Callable] = None,
        prediction_intervals: Optional[PredictionIntervals] = None,
        level: Optional[List[Union[int, float]]] = None,
        input_size: Optional[int] = None,
        fitted: bool = False,
        as_numpy: bool = False,
    ) -> DataFrame:
        """Perform time series cross validation.
        Creates `n_windows` splits where each window has `h` test periods, 
        trains the models, computes the predictions and merges the actuals.
        
        Parameters
        ----------
        df : pandas or polars DataFrame
            Series data in long format.
        n_windows : int
            Number of windows to evaluate.
        h : int
            Forecast horizon.
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        step_size : int, optional (default=None)
            Step size between each cross validation window. If None it will be equal to `h`.
        static_features : list of str, optional (default=None)
            Names of the features that are static and will be repeated when forecasting.
        dropna : bool (default=True)
            Drop rows with missing values produced by the transformations.
        keep_last_n : int, optional (default=None)
            Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it.
        max_horizon: int, optional (default=None)
            Train this many models, where each model will predict a specific horizon.            
        refit : bool or int (default=True)
            Retrain model for each cross validation window.
            If False, the models are trained at the beginning and then used to predict each window.
            If positive int, the models are retrained every `refit` windows.
        before_predict_callback : callable, optional (default=None)
            Function to call on the features before computing the predictions.
                This function will take the input dataframe that will be passed to the model for predicting and should return a dataframe with the same structure.
                The series identifier is on the index.
        after_predict_callback : callable, optional (default=None)
            Function to call on the predictions before updating the targets.
                This function will take a pandas Series with the predictions and should return another one with the same structure.
                The series identifier is on the index.
        prediction_intervals : PredictionIntervals, optional (default=None)
            Configuration to calibrate prediction intervals (Conformal Prediction).
        level : list of ints or floats, optional (default=None)
            Confidence levels between 0 and 100 for prediction intervals.
        input_size : int, optional (default=None)
            Maximum training samples per serie in each window. If None, will use an expanding window.
        fitted : bool (default=False)
            Store the in-sample predictions.
        as_numpy : bool (default = False)
            Cast features to numpy array.

        Returns
        -------
        result : pandas or polars DataFrame
            Predictions for each window with the series id, timestamp, last train date, target value and predictions from each model.
        """
        results = []
        self.cv_models_ = []
        self.ts._validate_freq(df, time_col)
        splits = backtest_splits(
            df,
            n_windows=n_windows,
            h=h,
            id_col=id_col,
            time_col=time_col,
            freq=self.freq,
            step_size=step_size,
            input_size=input_size,
        )
        self.cv_fitted_values_ = []
        for i_window, (cutoffs, train, valid) in enumerate(splits):
            should_fit = i_window == 0 or (refit > 0 and i_window % refit == 0)
            if should_fit:
                self.fit(
                    train,
                    id_col=id_col,
                    time_col=time_col,
                    target_col=target_col,
                    static_features=static_features,
                    dropna=dropna,
                    keep_last_n=keep_last_n,
                    max_horizon=max_horizon,
                    prediction_intervals=prediction_intervals,
                    fitted=fitted,
                    as_numpy=as_numpy,
                )
                self.cv_models_.append(self.models_)
                if fitted:
                    self.cv_fitted_values_.append(assign_columns(self.fcst_fitted_values_, 'fold', i_window))
            if fitted and not should_fit:
                if self.ts.target_transforms is not None:
                    for tfm in self.ts.target_transforms:
                        if hasattr(tfm, 'store_fitted'):
                            tfm.store_fitted = True
                prep = self.preprocess(
                    train,
                    id_col=id_col,
                    time_col=time_col,
                    target_col=target_col,
                    static_features=static_features,
                    dropna=dropna,
                    keep_last_n=keep_last_n,
                    max_horizon=max_horizon,
                    return_X_y=False,
                )
                assert not isinstance(prep, tuple)
                base = prep[[id_col, time_col]]
                train_X, train_y = self._extract_X_y(prep, target_col)
                if as_numpy:
                    train_X = to_numpy(train_X)
                del prep
                fitted_values = self._compute_fitted_values(
                    base=base,
                    X=train_X,
                    y=train_y,
                    id_col=id_col,
                    time_col=time_col,
                    target_col=target_col,
                    max_horizon=max_horizon,
                )
                fitted_values = assign_columns(fitted_values, 'fold', i_window)
                self.cv_fitted_values_.append(fitted_values)
            static = [c for c in self.ts.static_features_.columns if c != id_col]
            dynamic = [
                c for c in valid.columns if c not in static + [id_col, time_col, target_col]
            ]
            if dynamic:
                X_df: Optional[DataFrame] = valid.drop(columns=static + [target_col])
            else:
                X_df = None
            y_pred = self.predict(
                h=h,
                before_predict_callback=before_predict_callback,
                after_predict_callback=after_predict_callback,
                new_df=train if not should_fit else None,
                level=level,
                X_df=X_df,
            )
            y_pred = join(y_pred, cutoffs, on=id_col, how='left')
            result = join(
                valid[[id_col, time_col, target_col]],
                y_pred,
                on=[id_col, time_col],
            )
            sort_idxs = maybe_compute_sort_indices(result, id_col, time_col)
            if sort_idxs is not None:
                result = take_rows(result, sort_idxs)
            if result.shape[0] < valid.shape[0]:
                raise ValueError(
                    "Cross validation result produced less results than expected. "
                    "Please verify that the frequency set on the MLForecast constructor matches your series' "
                    "and that there aren't any missing periods."
                )
            results.append(result)
        del self.models_
        out = vertical_concat(results, match_categories=False)
        out = drop_index_if_pandas(out)
        first_out_cols = [id_col, time_col, 'cutoff', target_col]
        remaining_cols = [c for c in out.columns if c not in first_out_cols]
        return out[first_out_cols + remaining_cols]

    def cross_validation_fitted_values(self):
        if not getattr(self, 'cv_fitted_values_', []):
            raise ValueError('Please run cross_validation with fitted=True first.')
        out = vertical_concat(self.cv_fitted_values_, match_categories=False)            
        first_out_cols = [self.ts.id_col, self.ts.time_col, 'fold', self.ts.target_col]        
        remaining_cols = [c for c in out.columns if c not in first_out_cols]
        out = drop_index_if_pandas(out)
        return out[first_out_cols + remaining_cols]

### Data
This shows an example with just 4 series of the M4 dataset. If you want to run it yourself on all of them, you can refer to [this notebook](https://www.kaggle.com/code/lemuz90/m4-competition).

In [ ]:
import random

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
from datasetsforecast.m4 import M4, M4Info
from sklearn.linear_model import LinearRegression
from utilsforecast.plotting import plot_series
from window_ops.ewm import ewm_mean
from window_ops.expanding import expanding_mean
from window_ops.rolling import rolling_mean

from mlforecast.lgb_cv import LightGBMCV
from mlforecast.target_transforms import Differences, LocalStandardScaler
from mlforecast.utils import generate_daily_series

In [ ]:
group = 'Hourly'
await M4.async_download('data', group=group)
df, *_ = M4.load(directory='data', group=group)
df['ds'] = df['ds'].astype('int')
ids = df['unique_id'].unique()
random.seed(0)
sample_ids = random.choices(ids, k=4)
sample_df = df[df['unique_id'].isin(sample_ids)]
sample_df

,unique_id,ds,y
86796,H196,1,11.8
86797,H196,2,11.4
86798,H196,3,11.1
86799,H196,4,10.8
86800,H196,5,10.6
...,...,...,...
325235,H413,1004,99.0
325236,H413,1005,88.0
325237,H413,1006,47.0
325238,H413,1007,41.0


We now split this data into train and validation.

In [ ]:
info = M4Info[group]
horizon = info.horizon
valid = sample_df.groupby('unique_id').tail(horizon)
train = sample_df.drop(valid.index)
train.shape, valid.shape

((3840, 3), (192, 3))

In [ ]:
show_doc(MLForecast)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L137){target="_blank" style="float:right; font-size:smaller"}

### MLForecast

>      MLForecast (models:Union[sklearn.base.BaseEstimator,List[sklearn.base.Bas
>                  eEstimator],Dict[str,sklearn.base.BaseEstimator]],
>                  freq:Union[int,str,pandas._libs.tslibs.offsets.BaseOffset],
>                  lags:Optional[Iterable[int]]=None, lag_transforms:Optional[Di
>                  ct[int,List[Union[Callable,Tuple[Callable,Any]]]]]=None,
>                  date_features:Optional[Iterable[Union[str,Callable]]]=None,
>                  num_threads:int=1, target_transforms:Optional[List[Union[mlfo
>                  recast.target_transforms.BaseTargetTransform,mlforecast.targe
>                  t_transforms.BaseGroupedArrayTargetTransform]]]=None,
>                  transforms_namer:Optional[Callable]=None)

Forecasting pipeline

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| models | Union |  | Models that will be trained and used to compute the forecasts. |
| freq | Union |  | Pandas offset, pandas offset alias, e.g. 'D', 'W-THU' or integer denoting the frequency of the series. |
| lags | Optional | None | Lags of the target to use as features. |
| lag_transforms | Optional | None | Mapping of target lags to their transformations. |
| date_features | Optional | None | Features computed from the dates. Can be pandas date attributes or functions that will take the dates as input. |
| num_threads | int | 1 | Number of threads to use when computing the features. |
| target_transforms | Optional | None | Transformations that will be applied to the target before computing the features and restored after the forecasting step. |
| transforms_namer | Optional | None | Function that takes a transformation (either function or class), a lag and extra arguments and produces a name. |

The MLForecast object encapsulates the feature engineering + training the models + forecasting

In [ ]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)
fcst

MLForecast(models=[LGBMRegressor], freq=1, lag_features=['lag24', 'lag48', 'lag72', 'lag96', 'lag120', 'lag144', 'lag168', 'ewm_mean_lag48_alpha0.3'], date_features=[], num_threads=1)

Once we have this setup we can compute the features and fit the model.

In [ ]:
show_doc(MLForecast.fit)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L443){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.fit

>      MLForecast.fit
>                      (df:Union[pandas.core.frame.DataFrame,polars.dataframe.fr
>                      ame.DataFrame], id_col:str='unique_id',
>                      time_col:str='ds', target_col:str='y',
>                      static_features:Optional[List[str]]=None,
>                      dropna:bool=True, keep_last_n:Optional[int]=None,
>                      max_horizon:Optional[int]=None, prediction_intervals:Opti
>                      onal[mlforecast.utils.PredictionIntervals]=None,
>                      fitted:bool=False, as_numpy:bool=False)

Apply the feature engineering and train the models.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Series data in long format. |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target. |
| static_features | Optional | None | Names of the features that are static and will be repeated when forecasting.<br>    If `None`, will consider all columns (except id_col and time_col) as static. |
| dropna | bool | True | Drop rows with missing values produced by the transformations. |
| keep_last_n | Optional | None | Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it. |
| max_horizon | Optional | None | Train this many models, where each model will predict a specific horizon. |
| prediction_intervals | Optional | None | Configuration to calibrate prediction intervals (Conformal Prediction). |
| fitted | bool | False | Save in-sample predictions. |
| as_numpy | bool | False | Cast features to numpy array. |
| **Returns** | **MLForecast** |  | **Forecast object with series values and trained models.** |

In [ ]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)

In [ ]:
fcst.fit(train, fitted=True);

In [ ]:
show_doc(MLForecast.forecast_fitted_values)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L544){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.forecast_fitted_values

>      MLForecast.forecast_fitted_values ()

Access in-sample predictions.

In [ ]:
fcst.forecast_fitted_values()

,unique_id,ds,y,LGBMRegressor
0,H196,193,12.7,12.671271
1,H196,194,12.3,12.271271
2,H196,195,11.9,11.871271
3,H196,196,11.7,11.671271
4,H196,197,11.4,11.471271
...,...,...,...,...
3067,H413,956,59.0,68.280574
3068,H413,957,58.0,70.427570
3069,H413,958,53.0,44.767965
3070,H413,959,38.0,48.691257


In [ ]:
#| hide
# check that the fitted target from the fitted_values matches the original target after applying the transformation
fcst2 = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24]), LocalStandardScaler()],
)
fcst2.fit(train, fitted=True)
fitted_vals = fcst2.forecast_fitted_values()
train_restored = train.merge(
    fitted_vals.drop(columns='LGBMRegressor'),
    on=['unique_id', 'ds'],
    suffixes=('_expected', '_actual')
)
np.testing.assert_allclose(
    train_restored['y_expected'].values,
    train_restored['y_actual'].values,
)

# check fitted + max_horizon
max_horizon = 7
fcst2.fit(train, fitted=True, max_horizon=max_horizon)
max_horizon_fitted_values = fcst2.forecast_fitted_values()
# h is 1 to max_horizon
np.testing.assert_equal(
    np.sort(max_horizon_fitted_values['h'].unique()),
    np.arange(1, max_horizon + 1),
)
# predictions for the first horizon are equal to the recursive
pd.testing.assert_frame_equal(
    fitted_vals.reset_index(drop=True),
    max_horizon_fitted_values[max_horizon_fitted_values['h'] == 1].drop(columns='h'),
)
# restored values match
xx = max_horizon_fitted_values[lambda x: x['unique_id'].eq('H413')].pivot_table(
    index=['unique_id', 'ds'], columns='h', values='y'
).loc['H413']
first_ds = xx.index.min()
last_ds = xx.index.max()
for h in range(1, max_horizon):
    np.testing.assert_allclose(
        xx.loc[first_ds + h :, 1].values,
        xx.loc[: last_ds - h, h + 1].values,
    )

Once we've run this we're ready to compute our predictions.

In [ ]:
show_doc(MLForecast.predict)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L550){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.predict

>      MLForecast.predict (h:int,
>                          before_predict_callback:Optional[Callable]=None,
>                          after_predict_callback:Optional[Callable]=None, new_d
>                          f:Union[pandas.core.frame.DataFrame,polars.dataframe.
>                          frame.DataFrame,NoneType]=None,
>                          level:Optional[List[Union[int,float]]]=None, X_df:Uni
>                          on[pandas.core.frame.DataFrame,polars.dataframe.frame
>                          .DataFrame,NoneType]=None,
>                          ids:Optional[List[str]]=None)

Compute the predictions for the next `h` steps.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| h | int |  | Number of periods to predict. |
| before_predict_callback | Optional | None | Function to call on the features before computing the predictions.<br>    This function will take the input dataframe that will be passed to the model for predicting and should return a dataframe with the same structure.<br>    The series identifier is on the index. |
| after_predict_callback | Optional | None | Function to call on the predictions before updating the targets.<br>    This function will take a pandas Series with the predictions and should return another one with the same structure.<br>    The series identifier is on the index.    |
| new_df | Union | None | Series data of new observations for which forecasts are to be generated. <br>    This dataframe should have the same structure as the one used to fit the model, including any features and time series data. <br>    If `new_df` is not None, the method will generate forecasts for the new observations. |
| level | Optional | None | Confidence levels between 0 and 100 for prediction intervals. |
| X_df | Union | None | Dataframe with the future exogenous features. Should have the id column and the time column. |
| ids | Optional | None | List with subset of ids seen during training for which the forecasts should be computed.         |
| **Returns** | **Union** |  | **Predictions for each serie and timestep, with one column per model.** |

In [ ]:
predictions = fcst.predict(horizon)

We can see at a couple of results.

In [ ]:
results = valid.merge(predictions, on=['unique_id', 'ds'])
fig = plot_series(train, results, max_insample_length=0)
fig.savefig('figs/forecast__predict.png', bbox_inches='tight')

![](figs/forecast__predict.png)

In [ ]:
#| hide
# test new_df argument
pd.testing.assert_frame_equal(
    fcst.predict(horizon, new_df=train),
    predictions
)

#### Prediction intervals

With `MLForecast`, you can generate prediction intervals using Conformal Prediction. To configure Conformal Prediction, you need to pass an instance of the `PredictionIntervals` class to the `prediction_intervals` argument of the `fit` method. The class takes three parameters: `n_windows`, `h` and `method`.

* `n_windows` represents the number of cross-validation windows used to calibrate the intervals
* `h` is the forecast horizon
* `method` can be `conformal_distribution` or `conformal_error`; `conformal_distribution` (default) creates forecasts paths based on the cross-validation errors and calculate quantiles using those paths, on the other hand `conformal_error` calculates the error quantiles to produce prediction intervals. The strategy will adjust the intervals for each horizon step, resulting in different widths for each step. Please note that a minimum of 2 cross-validation windows must be used.

In [ ]:
fcst.fit(
    train,
    prediction_intervals=PredictionIntervals(n_windows=3, h=48)
);

After that, you just have to include your desired confidence levels to the `predict` method using the `level` argument. Levels must lie between 0 and 100.

In [ ]:
predictions_w_intervals = fcst.predict(48, level=[50, 80, 95])

In [ ]:
predictions_w_intervals.head()

,unique_id,ds,LGBMRegressor,LGBMRegressor-lo-95,LGBMRegressor-lo-80,LGBMRegressor-lo-50,LGBMRegressor-hi-50,LGBMRegressor-hi-80,LGBMRegressor-hi-95
0,H196,961,16.071271,15.958042,15.971271,16.005091,16.137452,16.171271,16.184501
1,H196,962,15.671271,15.553632,15.553632,15.578632,15.763911,15.788911,15.788911
2,H196,963,15.271271,15.153632,15.153632,15.162452,15.380091,15.388911,15.388911
3,H196,964,14.971271,14.858042,14.871271,14.905091,15.037452,15.071271,15.084501
4,H196,965,14.671271,14.553632,14.553632,14.562452,14.780091,14.788911,14.788911


In [ ]:
#| head
# test we can forecast horizon lower than h 
# with prediction intervals
for method in ['conformal_distribution', 'conformal_errors']:
    fcst.fit(
        train, 
        prediction_intervals=PredictionIntervals(n_windows=3, h=48)
    )

    preds_h_lower_h = fcst.predict(1, level=[50, 80, 95])
    preds_h_lower_h = fcst.predict(30, level=[50, 80, 95])

    # test monotonicity of intervals
    test_eq(
        preds_h_lower_h.filter(regex='lo|hi').apply(
            lambda x: x.is_monotonic_increasing,
            axis=1
        ).sum(),
        len(preds_h_lower_h)
    )

In [ ]:
#| hide
test_fail(lambda: fcst.predict(49, level=[68]))

In [ ]:
#| hide
# test we can recover point forecasts
test_eq(
    predictions,
    predictions_w_intervals[predictions.columns]
)

In [ ]:
#| hide
# test we can recover mean forecasts with level 0
test_close(
    predictions['LGBMRegressor'].values,
    fcst.predict(48, level=[0])['LGBMRegressor-lo-0'].values,
)

In [ ]:
#| hide
# test monotonicity of intervals
test_eq(
    predictions_w_intervals.filter(regex='lo|hi').apply(
        lambda x: x.is_monotonic_increasing,
        axis=1
    ).sum(),
    len(predictions_w_intervals)
)

Let's explore the generated intervals.

In [ ]:
results = valid.merge(predictions_w_intervals, on=['unique_id', 'ds'])
fig = plot_series(train, results, max_insample_length=0, level=[50, 80, 95])
fig.savefig('figs/forecast__predict_intervals.png', bbox_inches='tight')

![](figs/forecast__predict_intervals.png)

If you want to reduce the computational time and produce intervals with the same width for the whole forecast horizon, simple pass `h=1` to the `PredictionIntervals` class. The caveat of this strategy is that in some cases, variance of the absolute residuals maybe be small (even zero), so the intervals may be too narrow.

In [ ]:
fcst.fit(
    train,  
    prediction_intervals=PredictionIntervals(n_windows=3, h=1)
);

In [ ]:
predictions_w_intervals_ws_1 = fcst.predict(48, level=[80, 90, 95])

Let's explore the generated intervals.

In [ ]:
results = valid.merge(predictions_w_intervals_ws_1, on=['unique_id', 'ds'])
fig = plot_series(train, results, max_insample_length=0, level=[90])
fig.savefig('figs/forecast__predict_intervals_window_size_1.png', bbox_inches='tight')

![](figs/forecast__predict_intervals_window_size_1.png)

In [ ]:
#| hide
# test indexed data, datetime ds
fcst_test = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq='D',
    lags=[1],
    num_threads=1,
)
df_test = generate_daily_series(1)
fcst_test.fit(
    df_test,
    # prediction_intervals=PredictionIntervals()
)
pred_test = fcst_test.predict(12)
pred_int_test = fcst_test.predict(12, level=[80, 90])
# test same structure
test_eq(
    pred_test,
    pred_int_test[pred_test.columns]
)
# test monotonicity of intervals
test_eq(
    pred_int_test.filter(regex='lo|hi').apply(
        lambda x: x.is_monotonic_increasing,
        axis=1
    ).sum(),
    len(pred_int_test)
)

#### Forecast using a pretrained model

MLForecast allows you to use a pretrained model to generate forecasts for a new dataset. Simply provide a pandas dataframe containing the new observations as the value for the `new_df` argument when calling the `predict` method. The dataframe should have the same structure as the one used to fit the model, including any features and time series data. The function will then use the pretrained model to generate forecasts for the new observations. This allows you to easily apply a pretrained model to a new dataset and generate forecasts without the need to retrain the model.

In [ ]:
ercot_df = pd.read_csv('https://datasets-nixtla.s3.amazonaws.com/ERCOT-clean.csv')
# we have to convert the ds column to integers
# since MLForecast was trained with that structure
ercot_df['ds'] = np.arange(1, len(ercot_df) + 1)
# use the `new_df` argument to pass the ercot dataset 
ercot_fcsts = fcst.predict(horizon, new_df=ercot_df)
fig = plot_series(ercot_df, ercot_fcsts, max_insample_length=48 * 2)

In [ ]:
#| hide
fig.get_axes()[0].set_title('ERCOT forecasts trained on M4-Hourly dataset')
fig.savefig('figs/forecast__ercot.png', bbox_inches='tight')

![](figs/forecast__ercot.png)

If you want to take a look at the data that will be used to train the models you can call `Forecast.preprocess`.

In [ ]:
show_doc(MLForecast.preprocess)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L212){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.preprocess

>      MLForecast.preprocess
>                             (df:Union[pandas.core.frame.DataFrame,polars.dataf
>                             rame.frame.DataFrame], id_col:str='unique_id',
>                             time_col:str='ds', target_col:str='y',
>                             static_features:Optional[List[str]]=None,
>                             dropna:bool=True, keep_last_n:Optional[int]=None,
>                             max_horizon:Optional[int]=None,
>                             return_X_y:bool=False, as_numpy:bool=False)

Add the features to `data`.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Series data in long format. |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target. |
| static_features | Optional | None | Names of the features that are static and will be repeated when forecasting. |
| dropna | bool | True | Drop rows with missing values produced by the transformations. |
| keep_last_n | Optional | None | Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it. |
| max_horizon | Optional | None | Train this many models, where each model will predict a specific horizon. |
| return_X_y | bool | False | Return a tuple with the features and the target. If False will return a single dataframe. |
| as_numpy | bool | False | Cast features to numpy array. Only works for `return_X_y=True`. |
| **Returns** | **Union** |  | **`df` plus added features and target(s).** |

In [ ]:
prep_df = fcst.preprocess(train)
prep_df

,unique_id,ds,y,lag24,lag48,lag72,lag96,lag120,lag144,lag168,ewm_mean_lag48_alpha0.3
86988,H196,193,0.1,0.0,0.0,0.0,0.3,0.1,0.1,0.3,0.002810
86989,H196,194,0.1,-0.1,0.1,0.0,0.3,0.1,0.1,0.3,0.031967
86990,H196,195,0.1,-0.1,0.1,0.0,0.3,0.1,0.2,0.1,0.052377
86991,H196,196,0.1,0.0,0.0,0.0,0.3,0.2,0.1,0.2,0.036664
86992,H196,197,0.0,0.0,0.0,0.1,0.2,0.2,0.1,0.2,0.025665
...,...,...,...,...,...,...,...,...,...,...,...
325187,H413,956,0.0,10.0,1.0,6.0,-53.0,44.0,-21.0,21.0,7.963225
325188,H413,957,9.0,10.0,10.0,-7.0,-46.0,27.0,-19.0,24.0,8.574257
325189,H413,958,16.0,8.0,5.0,-9.0,-36.0,32.0,-13.0,8.0,7.501980
325190,H413,959,-3.0,17.0,-7.0,2.0,-31.0,22.0,5.0,-2.0,3.151386


If we do this we then have to call `Forecast.fit_models`, since this only stores the series information.

In [ ]:
#| hide
# transforms namer
def tfms_namer(tfm, lag, *args):
    return f'hello_from_{tfm.__name__}'

fcst2 = MLForecast(
    models=LinearRegression(),
    freq=1,
    lag_transforms={1: [expanding_mean]}, 
    transforms_namer=tfms_namer,
)
prep = fcst2.preprocess(train)
assert 'hello_from_expanding_mean' in prep

In [ ]:
show_doc(MLForecast.fit_models)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L268){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.fit_models

>      MLForecast.fit_models (X:Union[pandas.core.frame.DataFrame,polars.datafra
>                             me.frame.DataFrame,numpy.ndarray],
>                             y:numpy.ndarray)

Manually train models. Use this if you called `MLForecast.preprocess` beforehand.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| X | Union | Features. |
| y | ndarray | Target. |
| **Returns** | **MLForecast** | **Forecast object with trained models.** |

In [ ]:
X, y = prep_df.drop(columns=['unique_id', 'ds', 'y']), prep_df['y']
fcst.fit_models(X, y)

MLForecast(models=[LGBMRegressor], freq=1, lag_features=['lag24', 'lag48', 'lag72', 'lag96', 'lag120', 'lag144', 'lag168', 'ewm_mean_lag48_alpha0.3'], date_features=[], num_threads=1)

In [ ]:
predictions2 = fcst.predict(horizon)
pd.testing.assert_frame_equal(predictions, predictions2)

In [ ]:
#| hide
# test intervals multioutput
max_horizon = 24
individual_fcst_intervals = fcst.fit(
    train,
    max_horizon=max_horizon,
    prediction_intervals=PredictionIntervals(h=max_horizon)
)
individual_preds = fcst.predict(max_horizon)
individual_preds_intervals = fcst.predict(max_horizon, level=[90, 80])
# test monotonicity of intervals
test_eq(
    individual_preds_intervals.filter(regex='lo|hi').apply(
        lambda x: x.is_monotonic_increasing,
        axis=1
    ).sum(),
    len(individual_preds_intervals)
)
# test we can recover point forecasts with intervals
test_eq(
    individual_preds,
    individual_preds_intervals[individual_preds.columns]
)

In [ ]:
#| hide
# check for bad max_horizon & models_ states before predict
fcst = MLForecast(
    models=[LinearRegression()],
    freq=1,
    lags=[12],
)
fcst.fit(df, max_horizon=2)
fcst.preprocess(df, max_horizon=None)
test_fail(lambda: fcst.predict(1), contains='Found one model per horizon')

fcst.fit(df, max_horizon=None)
fcst.preprocess(df, max_horizon=2)
test_fail(lambda: fcst.predict(1), contains='Found a single model for all horizons')

In [ ]:
show_doc(MLForecast.cross_validation)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L689){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.cross_validation

>      MLForecast.cross_validation
>                                   (df:Union[pandas.core.frame.DataFrame,polars
>                                   .dataframe.frame.DataFrame], n_windows:int,
>                                   h:int, id_col:str='unique_id',
>                                   time_col:str='ds', target_col:str='y',
>                                   step_size:Optional[int]=None,
>                                   static_features:Optional[List[str]]=None,
>                                   dropna:bool=True,
>                                   keep_last_n:Optional[int]=None,
>                                   refit:Union[bool,int]=True,
>                                   max_horizon:Optional[int]=None, before_predi
>                                   ct_callback:Optional[Callable]=None, after_p
>                                   redict_callback:Optional[Callable]=None, pre
>                                   diction_intervals:Optional[mlforecast.utils.
>                                   PredictionIntervals]=None,
>                                   level:Optional[List[Union[int,float]]]=None,
>                                   input_size:Optional[int]=None,
>                                   fitted:bool=False, as_numpy:bool=False)

Perform time series cross validation.
Creates `n_windows` splits where each window has `h` test periods, 
trains the models, computes the predictions and merges the actuals.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | Union |  | Series data in long format. |
| n_windows | int |  | Number of windows to evaluate. |
| h | int |  | Forecast horizon. |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target. |
| step_size | Optional | None | Step size between each cross validation window. If None it will be equal to `h`. |
| static_features | Optional | None | Names of the features that are static and will be repeated when forecasting. |
| dropna | bool | True | Drop rows with missing values produced by the transformations. |
| keep_last_n | Optional | None | Keep only these many records from each serie for the forecasting step. Can save time and memory if your features allow it. |
| refit | Union | True | Retrain model for each cross validation window.<br>If False, the models are trained at the beginning and then used to predict each window.<br>If positive int, the models are retrained every `refit` windows. |
| max_horizon | Optional | None |  |
| before_predict_callback | Optional | None | Function to call on the features before computing the predictions.<br>    This function will take the input dataframe that will be passed to the model for predicting and should return a dataframe with the same structure.<br>    The series identifier is on the index. |
| after_predict_callback | Optional | None | Function to call on the predictions before updating the targets.<br>    This function will take a pandas Series with the predictions and should return another one with the same structure.<br>    The series identifier is on the index. |
| prediction_intervals | Optional | None | Configuration to calibrate prediction intervals (Conformal Prediction). |
| level | Optional | None | Confidence levels between 0 and 100 for prediction intervals. |
| input_size | Optional | None | Maximum training samples per serie in each window. If None, will use an expanding window. |
| fitted | bool | False | Store the in-sample predictions. |
| as_numpy | bool | False | Cast features to numpy array. |
| **Returns** | **Union** |  | **Predictions for each window with the series id, timestamp, last train date, target value and predictions from each model.** |

If we would like to know how good our forecast will be for a specific model and set of features then we can perform cross validation. What cross validation does is take our data and split it in two parts, where the first part is used for training and the second one for validation. Since the data is time dependant we usually take the last *x* observations from our data as the validation set.

This process is implemented in `MLForecast.cross_validation`, which takes our data and performs the process described above for `n_windows` times where each window has `h` validation samples in it. For example, if we have 100 samples and we want to perform 2 backtests each of size 14, the splits will be as follows:

1. Train: 1 to 72. Validation: 73 to 86.
2. Train: 1 to 86. Validation: 87 to 100.

You can control the size between each cross validation window using the `step_size` argument. For example, if we have 100 samples and we want to perform 2 backtests each of size 14 and move one step ahead in each fold (`step_size=1`), the splits will be as follows:

1. Train: 1 to 85. Validation: 86 to 99.
2. Train: 1 to 86. Validation: 87 to 100.

You can also perform cross validation without refitting your models for each window by setting `refit=False`. This allows you to evaluate the performance of your models using multiple window sizes without having to retrain them each time. 

In [ ]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        1: [(rolling_mean, 24)],
        24: [(rolling_mean, 24)],
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)
cv_results = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    fitted=True,
)
cv_results

,unique_id,ds,cutoff,y,LGBMRegressor
0,H196,865,864,15.5,15.373393
1,H196,866,864,15.1,14.973393
2,H196,867,864,14.8,14.673393
3,H196,868,864,14.4,14.373393
4,H196,869,864,14.2,14.073393
...,...,...,...,...,...
379,H413,956,912,59.0,64.284167
380,H413,957,912,58.0,64.830429
381,H413,958,912,53.0,40.726851
382,H413,959,912,38.0,42.739657


Since we set `fitted=True` we can access the predictions for the training sets as well with the `cross_validation_fitted_values` method.

In [ ]:
fcst.cross_validation_fitted_values()

,unique_id,ds,fold,y,LGBMRegressor
0,H196,193,0,12.7,12.673393
1,H196,194,0,12.3,12.273393
2,H196,195,0,11.9,11.873393
3,H196,196,0,11.7,11.673393
4,H196,197,0,11.4,11.473393
...,...,...,...,...,...
5563,H413,908,1,49.0,50.620196
5564,H413,909,1,39.0,35.972331
5565,H413,910,1,29.0,29.359678
5566,H413,911,1,24.0,25.784563


In [ ]:
#| hide
# test fitted
fcst2 = MLForecast(
    models=lgb.LGBMRegressor(n_estimators=5, random_state=0, verbosity=-1),
    freq=1,
    lags=[24],
    num_threads=1,
)
_ = fcst2.cross_validation(
    train,
    n_windows=2,
    h=2,
    fitted=True,
    refit=False,
)
fitted_cv_results = fcst2.cross_validation_fitted_values()
train_with_cv_fitted_values = train.merge(fitted_cv_results, on=['unique_id', 'ds'], suffixes=('_expected', ''))
np.testing.assert_allclose(
    train_with_cv_fitted_values['y_expected'].values,
    train_with_cv_fitted_values['y'].values,
)

# test with max_horizon
_ = fcst2.cross_validation(
    train,
    n_windows=2,
    h=2,
    fitted=True,
    refit=False,    
    max_horizon=2,
)
max_horizon_fitted_cv_results = fcst2.cross_validation_fitted_values()
pd.testing.assert_frame_equal(
    fitted_cv_results[lambda df: df['fold'].eq(0)],
    (
        max_horizon_fitted_cv_results
        [lambda df: df['fold'].eq(0) & df['h'].eq(1)]
        .drop(columns='h')
        .reset_index(drop=True)
    ),
)

We can also compute prediction intervals by passing a configuration to `prediction_intervals` as well as values for the width through `levels`.

In [ ]:
cv_results_intervals = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    prediction_intervals=PredictionIntervals(h=horizon),
    level=[80, 90]
)
cv_results_intervals

,unique_id,ds,cutoff,y,LGBMRegressor,LGBMRegressor-lo-90,LGBMRegressor-lo-80,LGBMRegressor-hi-80,LGBMRegressor-hi-90
0,H196,865,864,15.5,15.373393,15.311379,15.316528,15.430258,15.435407
1,H196,866,864,15.1,14.973393,14.940556,14.940556,15.006230,15.006230
2,H196,867,864,14.8,14.673393,14.606230,14.606230,14.740556,14.740556
3,H196,868,864,14.4,14.373393,14.306230,14.306230,14.440556,14.440556
4,H196,869,864,14.2,14.073393,14.006230,14.006230,14.140556,14.140556
...,...,...,...,...,...,...,...,...,...
379,H413,956,912,59.0,64.284167,29.890099,34.371545,94.196788,98.678234
380,H413,957,912,58.0,64.830429,56.874572,57.827689,71.833169,72.786285
381,H413,958,912,53.0,40.726851,35.296195,35.846206,45.607495,46.157506
382,H413,959,912,38.0,42.739657,35.292153,35.807640,49.671674,50.187161


The `refit` argument allows us to control if we want to retrain the models in every window. It can either be:

* A boolean: True will retrain on every window and False only on the first one.
* A positive integer: The models will be trained on the first window and then every `refit` windows.

In [ ]:
fcst = MLForecast(
    models=LinearRegression(),
    freq=1,
    lags=[1, 24],
)
for refit, expected_models in zip([True, False, 2], [4, 1, 2]):
    fcst.cross_validation(
        train,
        n_windows=4,
        h=horizon,
        refit=refit,
    )
    test_eq(len(fcst.cv_models_), expected_models)

In [ ]:
#| hide
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        1: [(rolling_mean, 24)],
        24: [(rolling_mean, 24)],
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)
cv_results_no_refit = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    refit=False
)
# test we recover the same "metadata"
test_eq(
    cv_results_no_refit.drop(columns='LGBMRegressor'),
    cv_results.drop(columns='LGBMRegressor')
)
# test the first window has the same forecasts
first_cutoff = cv_results['cutoff'].iloc[0]
test_eq(
    cv_results_no_refit.query('cutoff == @first_cutoff'),
    cv_results.query('cutoff == @first_cutoff')
)
# test next windows have different forecasts
test_ne(
    cv_results_no_refit.query('cutoff != @first_cutoff'),
    cv_results.query('cutoff != @first_cutoff')
)

In [ ]:
#| hide
# cv with input_size
input_size = 300
cv_results_input_size = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    input_size=input_size,
)
series_lengths = np.diff(fcst.ts.ga.indptr)
unique_lengths = np.unique(series_lengths)
assert unique_lengths.size == 1
assert unique_lengths[0] == input_size

In [ ]:
#| hide
# one model per horizon
cv_results2 = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    max_horizon=horizon,
)
# the first entry per id and window is the same
pd.testing.assert_frame_equal(
    cv_results.groupby(['unique_id', 'cutoff']).head(1),
    cv_results2.groupby(['unique_id', 'cutoff']).head(1)
)
# the rest is different
test_fail(lambda: pd.testing.assert_frame_equal(cv_results, cv_results2))

In [ ]:
#| hide
# one model per horizon with prediction intervals
cv_results2_intervals = fcst.cross_validation(
    train,
    n_windows=2,
    h=horizon,
    step_size=horizon,
    max_horizon=horizon,
    prediction_intervals=PredictionIntervals(n_windows=2, h=horizon),
    level=[80, 90]
)
# the first entry per id and window is the same
pd.testing.assert_frame_equal(
    cv_results_intervals.groupby(['unique_id', 'cutoff']).head(1),
    cv_results2_intervals.groupby(['unique_id', 'cutoff']).head(1)
)
# the rest is different
test_fail(lambda: pd.testing.assert_frame_equal(cv_results_intervals, cv_results2_intervals))

In [ ]:
#| hide
# wrong frequency raises error
df_wrong_freq = pd.DataFrame({'ds': pd.to_datetime(['2020-01-02', '2020-02-02', '2020-03-02', '2020-04-02'])})
df_wrong_freq['unique_id'] = 'id1'
df_wrong_freq['y'] = 1
fcst_wrong_freq = MLForecast(
    models=[LinearRegression()],
    freq='MS',
    lags=[1],
)
test_fail(
    lambda: fcst_wrong_freq.cross_validation(df_wrong_freq, n_windows=1, h=1),
    contains='Cross validation result produced less results than expected',
)

In [ ]:
fig = plot_series(cv_results, cv_results.drop(columns='cutoff'), max_insample_length=0)

In [ ]:
#| hide
fig.savefig('figs/forecast__cross_validation.png', bbox_inches='tight')

![](figs/forecast__cross_validation.png)

In [ ]:
fig = plot_series(cv_results_intervals, cv_results_intervals.drop(columns='cutoff'), level=[90], max_insample_length=0)

In [ ]:
#| hide
fig.savefig('figs/forecast__cross_validation_intervals.png', bbox_inches='tight')

![](figs/forecast__cross_validation_intervals.png)

In [ ]:
show_doc(MLForecast.from_cv)

---

[source](https://github.com/Nixtla/mlforecast/blob/main/mlforecast/forecast.py#L198){target="_blank" style="float:right; font-size:smaller"}

### MLForecast.from_cv

>      MLForecast.from_cv (cv:mlforecast.lgb_cv.LightGBMCV)

Once you've found a set of features and parameters that work for your problem you can build a forecast object from it using `MLForecast.from_cv`, which takes the trained `LightGBMCV` object and builds an `MLForecast` object that will use the same features and parameters. Then you can call fit and predict as you normally would.

In [ ]:
cv = LightGBMCV(
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [(ewm_mean, 0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])]
)
hist = cv.fit(
    train,
    n_windows=2,
    h=horizon,
    params={'verbosity': -1},
)

[LightGBM] [Info] Start training from score 0.084340
[10] mape: 0.118569
[20] mape: 0.111506
[30] mape: 0.107314
[40] mape: 0.106089
[50] mape: 0.106630
Early stopping at round 50
Using best iteration: 40


In [ ]:
fcst = MLForecast.from_cv(cv)
assert cv.best_iteration_ == fcst.models['LGBMRegressor'].n_estimators

In [ ]:
fcst.fit(train)
fcst.predict(horizon)

,unique_id,ds,LGBMRegressor
0,H196,961,16.111079
1,H196,962,15.711079
2,H196,963,15.311079
3,H196,964,15.011079
4,H196,965,14.711079
...,...,...,...
187,H413,1004,92.722032
188,H413,1005,69.153603
189,H413,1006,68.811675
190,H413,1007,53.693346


In [ ]:
#| hide
series = generate_daily_series(100, equal_ends=True, n_static_features=2, static_as_categorical=False)
non_std_series = series.copy()
non_std_series['ds'] = non_std_series.groupby('unique_id').cumcount()
non_std_series = non_std_series.rename(columns={'unique_id': 'some_id', 'ds': 'time', 'y': 'value'})
models = [
    lgb.LGBMRegressor(n_jobs=1, random_state=0, verbosity=-1),
    xgb.XGBRegressor(n_jobs=1, random_state=0),
]
flow_params = dict(
    models=models,
    lags=[7],
    lag_transforms={
        1: [expanding_mean],
        7: [(rolling_mean, 14)]
    },
    num_threads=2,
)
fcst = MLForecast(freq=1, **flow_params)
non_std_preds = fcst.fit(non_std_series, id_col='some_id', time_col='time', target_col='value').predict(7)
non_std_preds = non_std_preds.rename(columns={'some_id': 'unique_id'})
fcst = MLForecast(freq='D', **flow_params)
preds = fcst.fit(series).predict(7)
pd.testing.assert_frame_equal(preds.drop(columns='ds'), non_std_preds.drop(columns='time'))

In [ ]:
#|hide
def test_cross_validation(data=non_std_series, add_exogenous=False):
    n_windows = 2
    h = 14
    fcst = MLForecast(lgb.LGBMRegressor(verbosity=-1), freq=1, lags=[7, 14])
    if add_exogenous:
        data = data.assign(ex1 = lambda x: np.arange(0, len(x)))
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=DeprecationWarning)
        backtest_results = fcst.cross_validation(
            df=data,
            n_windows=n_windows,
            h=h,
            id_col='some_id',
            time_col='time',
            target_col='value',
            static_features=['some_id', 'static_0', 'static_1'],
        )
    renamer = {'some_id': 'unique_id', 'time': 'ds', 'value': 'y'}
    backtest_results = backtest_results.rename(columns=renamer)
    renamed = data.rename(columns=renamer)
    manual_results = []
    for cutoff, train, valid in backtest_splits(renamed, n_windows, h, 'unique_id', 'ds', 1):
        fcst.fit(train, static_features=['unique_id', 'static_0', 'static_1'])
        if add_exogenous:
            X_df = valid.drop(columns=['y', 'static_0', 'static_1']).reset_index()
        else:
            X_df = None
        pred = fcst.predict(h, X_df=X_df)
        res = valid[['unique_id', 'ds', 'y']].copy()
        res = res.merge(cutoff, on='unique_id')
        res = res[['unique_id', 'ds', 'cutoff', 'y']].copy()
        manual_results.append(res.merge(pred, on=['unique_id', 'ds'], how='left'))
    manual_results = pd.concat(manual_results)
    pd.testing.assert_frame_equal(backtest_results, manual_results.reset_index(drop=True))
test_cross_validation()
test_cross_validation(add_exogenous=True)

In [ ]:
#| hide
# test short series in cv
series = generate_daily_series(
    n_series=100, min_length=20, max_length=51, equal_ends=True,
)
horizon = 10
n_windows = 4
fcst = MLForecast(models=[LinearRegression()], freq='D', lags=[1])
cv_res = fcst.cross_validation(series, h=horizon, n_windows=n_windows)
series_per_cutoff = cv_res.groupby('cutoff')['unique_id'].nunique()
series_sizes = series['unique_id'].value_counts().sort_index()
for i in range(4):
    test_eq(series_per_cutoff[i], series_sizes.gt((n_windows - i) * horizon).sum())

In [ ]:
#| hide
#| polars
from itertools import product

import polars as pl
from utilsforecast.processing import match_if_categorical

from mlforecast.utils import generate_prices_for_series

In [ ]:
#| hide
#| polars
horizon = 2
series_pl = generate_daily_series(
    10, n_static_features=2, static_as_categorical=False, equal_ends=True, engine='polars'
)
series_pd = generate_daily_series(
    10, n_static_features=2, static_as_categorical=False, equal_ends=True, engine='pandas'
)
series_pd = series_pd.rename(columns={'static_0': 'product_id'})
prices_pd = generate_prices_for_series(series_pd, horizon)
prices_pd['unique_id'] = prices_pd['unique_id'].astype(series_pd['unique_id'].dtype)
series_pd = series_pd.merge(prices_pd, on=['unique_id', 'ds'])

prices_pl = pl.from_pandas(prices_pd)
uids_series, uids_prices = match_if_categorical(series_pl['unique_id'], prices_pl['unique_id'])
series_pl = series_pl.with_columns(uids_series)
prices_pl = prices_pl.with_columns(uids_prices)
series_pl = series_pl.rename({'static_0': 'product_id'}).join(prices_pl, on=['unique_id', 'ds'])
permutation = np.random.choice(series_pl.shape[0], series_pl.shape[0], replace=False)
series_pl = series_pl[permutation]
series_pd = series_pd.iloc[permutation]

cfg = dict(
    models=[LinearRegression(), lgb.LGBMRegressor(verbosity=-1)],
    freq='1d',
    lags=[1, 2],
    lag_transforms={
        1: [expanding_mean],
        2: [expanding_mean],
    },
    date_features=['day', 'month', 'week', 'year'],
    target_transforms=[Differences([1, 2]), LocalStandardScaler()],
)
fit_kwargs = dict(
    fitted=True,
    prediction_intervals=PredictionIntervals(h=horizon),
    static_features=['product_id', 'static_1'],
)
predict_kwargs = dict(
    h=2,
    level=[80, 95],
)
horizons = [None, horizon]
as_np = [True, False]
for max_horizon, as_numpy in product(horizons, as_np):
    fcst_pl = MLForecast(**cfg)
    fcst_pl.fit(series_pl, max_horizon=max_horizon, as_numpy=as_numpy, **fit_kwargs)
    fitted_pl = fcst_pl.forecast_fitted_values()
    preds_pl = fcst_pl.predict(X_df=prices_pl, **predict_kwargs)
    preds_pl_subset = fcst_pl.predict(X_df=prices_pl, ids=fcst_pl.ts.uids[:2], **predict_kwargs)
    cv_pl = fcst_pl.cross_validation(series_pl, n_windows=2, h=horizon, fitted=True, as_numpy=as_numpy)
    cv_fitted_pl = fcst_pl.cross_validation_fitted_values()
    
    fcst_pd = MLForecast(**cfg)
    fcst_pd.fit(series_pd, max_horizon=max_horizon, as_numpy=as_numpy, **fit_kwargs)
    fitted_pd = fcst_pd.forecast_fitted_values()
    preds_pd = fcst_pd.predict(X_df=prices_pd, **predict_kwargs)
    preds_pd_subset = fcst_pd.predict(X_df=prices_pd, ids=fcst_pd.ts.uids[:2], **predict_kwargs)
    cv_pd = fcst_pd.cross_validation(series_pd, n_windows=2, h=horizon, fitted=True, as_numpy=as_numpy)
    cv_fitted_pd = fcst_pd.cross_validation_fitted_values()

    if max_horizon is not None:
        fitted_pl = fitted_pl.with_columns(pl.col('h').cast(pl.Int64))
        for h in range(max_horizon):
            fitted_h = fitted_pl.filter(pl.col('h').eq(h + 1))
            series_offset = (
                series_pl
                .sort('unique_id', 'ds')
                .with_columns(pl.col('y').shift(-h).over('unique_id'))
            )
            series_filtered = (
                fitted_h
                [['unique_id', 'ds']]
                .join(series_offset, on=['unique_id', 'ds'])
                .sort(['unique_id', 'ds'])
            )
            np.testing.assert_allclose(
                series_filtered['y'],
                fitted_h['y']
            )
    else:
        series_filtered = (
            fitted_pl
            [['unique_id', 'ds']]
            .join(series_pl, on=['unique_id', 'ds'])
            .sort(['unique_id', 'ds'])
        )
        np.testing.assert_allclose(
            series_filtered['y'],
            fitted_pl['y']
        )

    pd.testing.assert_frame_equal(fitted_pl.to_pandas(), fitted_pd)
    pd.testing.assert_frame_equal(preds_pl.to_pandas(), preds_pd)
    pd.testing.assert_frame_equal(preds_pl_subset.to_pandas(), preds_pd_subset)    
    pd.testing.assert_frame_equal(cv_pl.to_pandas(), cv_pd)
    pd.testing.assert_frame_equal(
        cv_fitted_pl.with_columns(pl.col('fold').cast(pl.Int64)).to_pandas(),
        cv_fitted_pd,
    )

In [ ]:
#| hide
# test transforms are inverted correctly when series were dropped
series = generate_daily_series(10, min_length=5, max_length=20)
fcst = MLForecast(
    models=LinearRegression(),
    freq='D',
    lags=[10],
    target_transforms=[Differences([1]), LocalStandardScaler()],
)
fcst.fit(series, fitted=True)
assert fcst.ts._dropped_series.size > 0
fitted_vals = fcst.forecast_fitted_values()
full = fitted_vals.merge(series, on=['unique_id', 'ds'], suffixes=('_fitted', '_orig'))
np.testing.assert_allclose(
    full['y_fitted'].values,
    full['y_orig'].values,
)